In [1]:
import pandas as pd
import numpy as np
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import os
import re
import pickle

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  0


## Load pre-trained translation model from fairseq.
You can always use another middle language or other models.

In [3]:
# Load translation model
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /Users/GYA/.cache/torch/hub/pytorch_fairseq_main
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-ru.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-ru.single_model.tar.gz from cache at /Users/GYA/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949d46ae3d0a06c1be3667929101011cdc0.8699fa0753ba14331352cc26ceb7aa46fba856bad52dd40fc97d3834418bf6f1
/Users/GYA/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary cod

In [4]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /Users/GYA/.cache/torch/hub/pytorch_fairseq_main
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /Users/GYA/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq.tasks.translation:[en] dictionary: 42024 types
INFO:fairseq.tasks.translation:[de] dictionary: 42024 types
INFO:fairseq.models.fairseq_model:{'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': 'simple', 'l

In [5]:
en2ru.cuda()
ru2en.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
en2de.cuda()
de2en.cuda()

## Load data

In [6]:
path = './'
train_df = pd.read_csv(path+'train.csv', header=None)
train_df.head()

,0,1,2
0,5,5,why doesn't an optical mouse work on a glass t...
1,6,6,what is the best off-road motorcycle trail ? l...
2,3,3,what is trans fat? how to reduce that? i heard...
3,7,7,how many planes fedex has? i heard that it is ...
4,7,7,"in the san francisco bay area, does it make se..."


In [7]:
train_labels = [v-1 for v in train_df[0]]
train_text = [v for v in train_df[2]]

In [8]:
train_text[0]

"why doesn't an optical mouse work on a glass table? or even on some surfaces? optical mice use an led and a camera to rapidly capture images of the surface beneath the mouse.  the infomation from the camera is analyzed by a dsp (digital signal processor) and used to detect imperfections in the underlying surface and determine motion. some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the dsp to accurately analyze the surface beneath the mouse.   since glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the dsp not to recognize motion properly. when the system is unable to see surface changes associated with movement, the mouse will not work properly."

In [9]:
len(train_text)

1400000

In [10]:
max(train_labels)

9

In [11]:
# split and get our unlabeled training data
def train_val_split(labels, n_labeled_per_class, n_labels, seed = 0):
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        train_labeled_idxs.extend(idxs[:n_labeled_per_class])
        train_unlabeled_idxs.extend(idxs[n_labeled_per_class : n_labeled_per_class + 10000])
        val_idxs.extend(idxs[-3000:])
    
    np.random.shuffle(train_labeled_idxs)
    np.random.shuffle(train_unlabeled_idxs)
    np.random.shuffle(val_idxs)
    return train_labeled_idxs, train_unlabeled_idxs, val_idxs

In [12]:
train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(train_labels, 500, 10)

In [13]:
len(train_unlabeled_idxs)

100000

In [14]:
idxs = train_unlabeled_idxs

In [15]:
idxs[0]

420562

## Back translation process
You can tune the temperature in the translation process to control the diversity.

In [16]:
# back translate using Russian as middle language
def translate_ru(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = ru2en.translate(en2ru.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [17]:
# back translate using German as middle language
def translate_de(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = de2en.translate(en2de.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [18]:
translate_de(0,100000, 'de_1.pkl')

/var/folders/lc/r27xvf5s7lv9dr0lxwrbnk9r0000gn/T/ipykernel_16487/3549724842.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm(range(start, end)):


  0%|          | 0/100000 [00:00<?, ?it/s]

INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1


KeyboardInterrupt: 